In [0]:
%sql
USE CATALOG hive_metastore;
USE SCHEMA default;

In [0]:
# Databricks Concurrency Write Conflict Test
# Test Setup: Create this notebook and run simultaneously from 2 different clusters

import time
import random
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime
import uuid

# Configuration
TABLE_NAME = "hive_metastore.default.concurrency_test_table"
CLUSTER_ID = str(uuid.uuid4())[:8]  # Unique identifier for this cluster

print(f"Starting test on cluster: {CLUSTER_ID}")
print(f"Target table: {TABLE_NAME}")
print(f"Timestamp: {datetime.now()}")

# Step 1: Create the test table (run this once from one cluster first)
def setup_test_table():
    """Setup the initial Delta table"""
    try:
        # Create initial table
        spark.sql(f"""
        CREATE OR REPLACE TABLE {TABLE_NAME} (
            id INT,
            cluster_id STRING,
            timestamp TIMESTAMP,
            operation_id STRING
        ) USING DELTA
        """)
        
        # Insert initial row
        spark.sql(f"""
        INSERT INTO {TABLE_NAME} 
        VALUES (1, 'initial', current_timestamp(), 'setup')
        """)
        
        print("✅ Test table created successfully")
    except Exception as e:
        print(f"❌ Table setup error: {e}")

setup_test_table()

Starting test on cluster: 64d79fc6
Target table: hive_metastore.default.concurrency_test_table
Timestamp: 2025-10-03 11:01:36.244943
✅ Test table created successfully


In [0]:
%sql
select * from hive_metastore.default.concurrency_test_table;

id,cluster_id,timestamp,operation_id
1,initial,2025-10-03T10:36:59.858165Z,setup
